### In this assignment, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
#### Run LASSO with different L1 penalties.
#### Choose best L1 penalty using a validation set.
#### Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log, sqrt
from sklearn.linear_model import Lasso
%matplotlib inline

In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [4]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [5]:
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


#### Create new features by performing following transformation on inputs:

In [7]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)

In [8]:
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms'] * sales['bedrooms']
sales['floors_square'] = sales['floors'] * sales['floors']

##### Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
##### On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

In [10]:
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_living_sqrt,sqft_lot_sqrt,bedrooms_square,floors_square
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,0,98178,47.5112,-122.257,1340.0,5650.0,34.351128,75.166482,9.0,1.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,1991,98125,47.7210,-122.319,1690.0,7639.0,50.695167,85.099941,9.0,4.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,0,98028,47.7379,-122.233,2720.0,8062.0,27.748874,100.000000,4.0,1.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,0,98136,47.5208,-122.393,1360.0,5000.0,44.271887,70.710678,16.0,1.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,0,98074,47.6168,-122.045,1800.0,7503.0,40.987803,89.888820,9.0,1.0


#### Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.

In [12]:
all_features = ['bedrooms', 'bedrooms_square','bathrooms', 'sqft_living', 'sqft_living_sqrt','sqft_lot', 
                'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade',
                'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

In [13]:
# set parameters
model_all = Lasso(alpha=5e2, normalize=True)

In [14]:
# learn weights
model_all.fit(sales[all_features], sales['price'])

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

#### Quiz Question: Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?

In [21]:
model_all.coef_

array([    0.        ,     0.        ,     0.        ,   134.43931396,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        , 24750.00458561,     0.        ,
       61749.10309071,     0.        ,     0.        ,    -0.        ,
           0.        ])

#### To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets. Download the provided csv files containing training, validation and test sets.

In [22]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [23]:
# Make sure to create the 4 features as we did in #1:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms'] * testing['bedrooms']
testing['floors_square'] = testing['floors'] * testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms'] * training['bedrooms']
training['floors_square'] = training['floors'] * training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms'] * validation['bedrooms']
validation['floors_square'] = validation['floors'] * validation['floors']

#### Now for each l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type np.logspace(1, 7, num=13).)
#### Learn a model on TRAINING data using the specified l1_penalty. Make sure to specify normalize=True in the constructor:

In [24]:
L1_penalty_list = np.logspace(1, 7, num=13)

In [36]:
def different_l1_penalty_lasso(train_data, validation_data, L1_penalty, all_features_name):
    """
    construct model with different L1_penalty and calculate RSS on validation data
    """
    model_lasso = Lasso(alpha=L1_penalty, normalize=True)
    model_lasso.fit(train_data[all_features_name], train_data['price'])
    RSS_valid = np.sum( (validation_data['price'] - model_lasso.predict(validation_data[all_features_name])) ** 2 )
    
    return RSS_valid

In [37]:
result = []
for i in L1_penalty_list:
    result.append([i, different_l1_penalty_lasso(training, validation, i, all_features)])

#### Quiz Question: Which was the best value for the l1_penalty, i.e. which value of l1_penalty produced the lowest RSS on VALIDATION data?

In [39]:
df = pd.DataFrame(result, columns=['L1_penalty', 'RSS'])

In [40]:
df

,L1_penalty,RSS
0,1.000000e+01,3.982133e+14
1,3.162278e+01,3.990419e+14
2,1.000000e+02,4.297916e+14
3,3.162278e+02,4.637398e+14
4,1.000000e+03,6.458987e+14
5,3.162278e+03,1.222507e+15
6,1.000000e+04,1.222507e+15
7,3.162278e+04,1.222507e+15
8,1.000000e+05,1.222507e+15
9,3.162278e+05,1.222507e+15


#### Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty.

In [50]:
L1_penalty_list[0]

10.0

In [49]:
print("RSS on TEST data for the model with best L1 penalty is: {}".
      format(different_l1_penalty_lasso(training, testing, 10, all_features)))

RSS on TEST data for the model with best L1 penalty is: 98467402552698.83


#### Quiz Question: Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero.

In [51]:
model_lasso_best_l1 = Lasso(alpha=10, normalize=True)

In [52]:
model_lasso_best_l1.fit(training[all_features], training['price'])

Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [56]:
np.count_nonzero(model_lasso_best_l1.coef_) + np.count_nonzero(model_lasso_best_l1.intercept_)

15

#### What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.
#### You are going to implement a simple, two phase procedure to achieve this goal:
##### Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.
##### Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for ‘l1_penalty’.

In [57]:
L1_penalty_list_large = np.logspace(1, 4, num=20)

In [61]:
def Get_num_coef_different_l1_penalty_lasso(train_data, L1_penalty, all_features_name):
    """
    construct model with different L1_penalty and calculate the number of nonzero coefficient (including intercept)
    """
    model_lasso = Lasso(alpha=L1_penalty, normalize=True)
    model_lasso.fit(train_data[all_features_name], train_data['price'])
    Num_non_zero_coef = np.count_nonzero(model_lasso.coef_) + np.count_nonzero(model_lasso.intercept_)
    
    return Num_non_zero_coef

In [62]:
Number_coeff = []

In [63]:
for i in L1_penalty_list_large:
    Number_coeff.append([i, Get_num_coef_different_l1_penalty_lasso(training, i, all_features)])

In [80]:
df_Number_coeff = pd.DataFrame(Number_coeff, columns=['L1_penalty_large', 'Number'])

In [81]:
df_Number_coeff

,L1_penalty_large,Number
0,10.000000,15
1,14.384499,15
2,20.691381,15
3,29.763514,15
4,42.813324,13
5,61.584821,12
6,88.586679,11
7,127.427499,10
8,183.298071,7
9,263.665090,6


#### Out of this large range, we want to find the two ends of our desired narrow range of l1_penalty. At one end, we will have l1_penalty values that have too few non-zeros, and at the other end, we will have an l1_penalty that has too many non-zeros.
##### The largest l1_penalty that has more non-zeros than ‘max_nonzeros’ (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)Store this value in the variable ‘l1_penalty_min’ (we will use it later)
##### The smallest l1_penalty that has fewer non-zeros than ‘max_nonzeros’ (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)Store this value in the variable ‘l1_penalty_max’ (we will use it later)

In [85]:
l1_penalty_min = df_Number_coeff[df_Number_coeff['Number'] > 7].iloc[-1]['L1_penalty_large']
l1_penalty_max = df_Number_coeff[df_Number_coeff['Number'] < 7].iloc[0]['L1_penalty_large']

#### What values did you find for l1_penalty_min and l1_penalty_max?

In [86]:
print(l1_penalty_min, l1_penalty_max)

127.42749857031335 263.6650898730358


In [87]:
L1_penalty_list_correct_range = np.linspace(l1_penalty_min, l1_penalty_max, 20)

#### For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
##### Fit a regression model with a given l1_penalty on TRAIN data. As before, use "alpha=l1_penalty" and "normalize=True".
##### Measure the RSS of the learned model on the VALIDATION set

In [90]:
def different_l1_penalty_lasso_small_range(train_data, validation_data, L1_penalty, all_features_name):
    """
    construct model with different L1_penalty
    calculate RSS on validation data and the number of nonzero coefficient (including intercept)
    """
    model_lasso = Lasso(alpha=L1_penalty, normalize=True)
    model_lasso.fit(train_data[all_features_name], train_data['price'])
    Num_non_zero_coef = np.count_nonzero(model_lasso.coef_) + np.count_nonzero(model_lasso.intercept_)
    
    RSS_valid = np.sum( (validation_data['price'] - model_lasso.predict(validation_data[all_features_name])) ** 2 )
    
    return [RSS_valid, Num_non_zero_coef]

#### Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. (Again, take account of the intercept when counting the number of nonzeros.)

In [92]:
result_max_nonzeros = []
for i in L1_penalty_list_correct_range:
    result_max_nonzeros.append([i] + different_l1_penalty_lasso_small_range(training, validation, i, all_features))

In [94]:
df_result_max_nonzeros = pd.DataFrame(result_max_nonzeros, columns=['L1_penalty', 'RSS', 'Number of Non_zero Coef'])

In [95]:
df_result_max_nonzeros

,L1_penalty,RSS,Number of Non_zero Coef
0,127.427499,4.353747e+14,10
1,134.597898,4.370092e+14,10
2,141.768298,4.382361e+14,8
3,148.938697,4.391589e+14,8
4,156.109097,4.400374e+14,7
5,163.279496,4.407775e+14,7
6,170.449896,4.415667e+14,7
7,177.620295,4.424064e+14,7
8,184.790695,4.432967e+14,7
9,191.961094,4.442398e+14,7


In [98]:
df_coef_7 = df_result_max_nonzeros[df_result_max_nonzeros['Number of Non_zero Coef'] == 7]

In [110]:
df_coef_7.sort_values('RSS')

,L1_penalty,RSS,Number of Non_zero Coef
4,156.109097,4.400374e+14,7
5,163.279496,4.407775e+14,7
6,170.449896,4.415667e+14,7
7,177.620295,4.424064e+14,7
8,184.790695,4.432967e+14,7
9,191.961094,4.442398e+14,7
10,199.131494,4.452307e+14,7


#### What features in this model have non-zero coefficients?

In [111]:
minimal_RSS_l1_penalty = df_coef_7.sort_values('RSS')['L1_penalty'].iloc[0]

In [112]:
model_minimal_RSS_7 = Lasso(alpha=minimal_RSS_l1_penalty, normalize=True)

In [113]:
model_minimal_RSS_7.fit(training[all_features], training['price'])

Lasso(alpha=156.10909673930755, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=True, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [116]:
pd.DataFrame(model_minimal_RSS_7.coef_, index=all_features, columns=['Coeff Value'])

,Coeff Value
bedrooms,-0.000000
bedrooms_square,-0.000000
bathrooms,10610.890284
sqft_living,163.380252
sqft_living_sqrt,0.000000
sqft_lot,-0.000000
sqft_lot_sqrt,-0.000000
floors,0.000000
floors_square,0.000000
waterfront,506451.687115
